# PROJET : Anticiper les besoins en consommation électrique de bâtiments

**Autor** : Louis BIRENHOLZ   
**Date** : 12/03/2020  
**Durée totale (3 notebooks)** : 100 heures   
**Vidéo du projet** : https://www.youtube.com/watch?v=GRcUu2_YbFE&feature=emb_title

Pour ce projet, nous nous intéressons aux objectifs de la ville de **Seattle** : **devenir une ville neutre en émissions de carbone en 2050**. On s’intéresse de près aux émissions des bâtiments non destinés à l’habitation.  

On souhaite prédire les émissions de CO2 et la consommation totale d’énergie de bâtiments pour lesquels elles n’ont pas encore été mesurées. Ces mesures sont chères et complexes à réaliser.

## Partie 1 : Cleaning & Preprocessing

Ce 1er notebook réalise un **cleaning des données**, du **feature engineering** et la création d'un **unique dataframe** pour la suite.  

## Librairies

In [2]:
# Classic lib.
import pandas as pd
import numpy as np

# Viz lib.
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from ast import literal_eval

## Settings 

In [2]:
# Style.
plt.style.use('ggplot')
sns.set_style("darkgrid")

# Option.
pd.options.display.max_columns = 100

## Data

In [3]:
# Données de 2015.
data2015 = pd.read_csv("2015-building-energy-benchmarking.csv", sep=',')

# Données de 2016.
data2016 = pd.read_csv("2016-building-energy-benchmarking.csv", sep=',')

In [4]:
# Dimensions.
print(data2015.shape)
print(data2016.shape)

(3340, 47)
(3376, 46)


In [5]:
# Check des lignes dupliquées.
data2015.drop_duplicates(inplace = True)
data2016.drop_duplicates(inplace = True)

print(data2015.shape)
print(data2016.shape)

(3340, 47)
(3376, 46)


## Quick check 2015 & 2016

In [22]:
data2015.head(3)

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),OtherFuelUse(kBtu),GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity(kgCO2e/ft2),DefaultData,ComplianceStatus,City Council Districts,SPD Beats,Zip Codes,Latitude,Longitude
0,1,2015,NonResidential,Hotel,MAYFLOWER PARK HOTEL,659000030,7,DOWNTOWN,1927,1,12.0,88434,0,88434,Hotel,Hotel,88434.0,NaN,NaN,NaN,NaN,NaN,65.0,78.9,80.3,173.5,175.1,6981428.0,7097539.0,2023032.0,1080307.0,3686160.0,12724.0,1272388.0,0.0,249.43,2.64,No,Compliant,NaN,31.0,18081,47.61219025,-122.33799744
1,2,2015,NonResidential,Hotel,PARAMOUNT HOTEL,659000220,7,DOWNTOWN,1996,1,11.0,103566,15064,88502,"Hotel, Parking, Restaurant",Hotel,83880.0,Parking,15064.0,Restaurant,4622.0,NaN,51.0,94.4,99.0,191.3,195.2,8354235.0,8765788.0,0.0,1144563.0,3905411.0,44490.0,4448985.0,0.0,263.51,2.38,No,Compliant,NaN,31.0,18081,47.61310583,-122.33335756
2,3,2015,NonResidential,Hotel,WESTIN HOTEL,659000475,7,DOWNTOWN,1969,1,41.0,961990,0,961990,"Hotel, Parking, Swimming Pool",Hotel,757243.0,Parking,100000.0,Swimming Pool,0.0,NaN,18.0,96.6,99.7,242.7,246.5,73130656.0,75506272.0,19660404.0,14583930.0,49762435.0,37099.0,3709900.0,0.0,2061.48,1.92,Yes,Compliant,NaN,31.0,18081,47.61334897,-122.33769944


In [23]:
data2016.head(3)

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,ComplianceStatus,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,0659000030,7,DOWNTOWN,47.61220,-122.33799,1927,1.0,12,88434,0,88434,Hotel,Hotel,88434.0,NaN,NaN,NaN,NaN,NaN,60.0,81.699997,84.300003,182.500000,189.000000,7226362.5,7456910.0,2003882.0,1.156514e+06,3946027.0,12764.52930,1276453.0,False,Compliant,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,0659000220,7,DOWNTOWN,47.61317,-122.33393,1996,1.0,11,103566,15064,88502,"Hotel, Parking, Restaurant",Hotel,83880.0,Parking,15064.0,Restaurant,4622.0,NaN,61.0,94.800003,97.900002,176.100006,179.399994,8387933.0,8664479.0,0.0,9.504252e+05,3242851.0,51450.81641,5145082.0,False,Compliant,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,0659000475,7,DOWNTOWN,47.61393,-122.33810,1969,1.0,41,956110,196718,759392,Hotel,Hotel,756493.0,NaN,NaN,NaN,NaN,NaN,43.0,96.000000,97.699997,241.899994,244.100006,72587024.0,73937112.0,21566554.0,1.451544e+07,49526664.0,14938.00000,1493800.0,False,Compliant,2089.28,2.19


## Cleaning 

In [9]:
# A t-on les mêmes colonnes dans les 2 dataframes ?

Col_absentes_A=[]
Col_absentes_B=[]

for i,j in zip(data2015.columns,data2016.columns):
    if i not in data2016.columns:
        Col_absentes_A.append(i)
    if j not in data2015.columns:
        Col_absentes_B.append(j)  
        
print(" Les colonnes présentent dans 2015 mais pas dans 2016 sont :",Col_absentes_A,".")
print("***************************")
print(print(" Les colonnes présentent dans 2016 mais pas dans 2015 sont :",Col_absentes_B,"."))

 Les colonnes présentent dans 2015 mais pas dans 2016 sont : ['Location', 'OtherFuelUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)', 'GHGEmissionsIntensity(kgCO2e/ft2)', 'Comment', '2010 Census Tracts', 'Seattle Police Department Micro Community Policing Plan Areas', 'City Council Districts', 'SPD Beats'] .
***************************
 Les colonnes présentent dans 2016 mais pas dans 2015 sont : ['Address', 'City', 'State', 'ZipCode', 'Latitude', 'Longitude', 'Comments', 'TotalGHGEmissions', 'GHGEmissionsIntensity'] .
None


## Check des NaN

In [10]:
# 2015.
df_2015_NaN = pd.DataFrame(((data2015.notnull().sum().sort_values(ascending=False))/np.shape(data2015)[0])*100)
df_2015_NaN.columns = ['% de données dispo']
df_2015_NaN

,% de données dispo
Zip Codes,100.000000
PropertyGFABuilding(s),100.000000
DataYear,100.000000
BuildingType,100.000000
PrimaryPropertyType,100.000000
PropertyName,100.000000
Location,100.000000
CouncilDistrictCode,100.000000
Neighborhood,100.000000
YearBuilt,100.000000


In [11]:
# 2016.
df_2016_NaN = pd.DataFrame(((data2016.notnull().sum().sort_values(ascending=False))/np.shape(data2016)[0])*100)
df_2016_NaN.columns = ['% de données dispo']
df_2016_NaN

,% de données dispo
OSEBuildingID,100.000000
PropertyGFABuilding(s),100.000000
PropertyGFATotal,100.000000
NumberofFloors,100.000000
YearBuilt,100.000000
Longitude,100.000000
Latitude,100.000000
Neighborhood,100.000000
CouncilDistrictCode,100.000000
TaxParcelIdentificationNumber,100.000000


## Check des outliers


Exploration de la colonne nommée **outlier**

In [12]:
data2015['Outlier'] = data2015['Outlier'].fillna(0)
data2016['Outlier'] = data2016['Outlier'].fillna(0)

mask1 = (data2015['Outlier'] != 0)
mask2 = (data2015['Outlier'] == 0)
mask3 = (data2016['Outlier'] == 0)

A = data2015[mask1]['GHGEmissions(MetricTonsCO2e)'].dropna()
B = data2015[mask2]['GHGEmissions(MetricTonsCO2e)'].dropna()

In [14]:
# Delete outliers.

data2015 = data2015[mask2]
data2016 = data2016[mask3]

# On retire la colonne Outlier.
del data2015['Outlier']
del data2016['Outlier']

In [15]:
print(type(data2015["Location"][0])) # Objet de type str.
data2015["Location"] = data2015["Location"].apply(literal_eval) # On change avec .apply(literal_eval) un string en dictionnaire.
print(type(data2015["Location"][0])) # Objet de type dictionnaire .

<class 'str'>
<class 'dict'>


In [16]:
data2015["Latitude"] = data2015["Location"].apply(lambda x : x["latitude"]) # On applique la clé "latitude" à l'ensemble de la colonne "location".
data2015["Longitude"] = data2015["Location"].apply(lambda x : x["longitude"]) 

In [17]:
# Destruction de colonnes inutiles pour la concaténation de 2015 et 2016.

data2015.drop(['Location','Comment','2010 Census Tracts',
               'Seattle Police Department Micro Community Policing Plan Areas'], axis = 1, inplace = True)
data2016.drop(['Address', 'City', 'State', 'ZipCode', 'Comments'],axis = 1, inplace = True)

In [18]:
# On retient seulement les bâtiments non résidentiels pour l'étude.

print(data2015['BuildingType'].unique())
print(data2016['BuildingType'].unique())

NonResidential_building = ['NonResidential','Nonresidential COS','Campus','Nonresidential WA']

['NonResidential' 'Nonresidential COS' 'Multifamily MR (5-9)'
 'SPS-District K-12' 'Multifamily LR (1-4)' 'Campus'
 'Multifamily HR (10+)']
['NonResidential' 'Nonresidential COS' 'Multifamily MR (5-9)'
 'SPS-District K-12' 'Campus' 'Multifamily LR (1-4)'
 'Multifamily HR (10+)' 'Nonresidential WA']


In [19]:
# On ne prend que les bâtiments non residentiels.

mask1 = (data2015['BuildingType'].isin(NonResidential_building))
mask2 = (data2016['BuildingType'].isin(NonResidential_building))

df2015 = data2015[mask1]
df2016 = data2016[mask2]

In [20]:
print(df2015['BuildingType'].unique())
print(df2016['BuildingType'].unique())

['NonResidential' 'Nonresidential COS' 'Campus']
['NonResidential' 'Nonresidential COS' 'Campus' 'Nonresidential WA']


In [21]:
# Concaténation des 2 dataframes 2015 et 2016 pour former data_tot.

df_tot = pd.concat([df2015,df2016],sort=False)
df_tot.head()

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),OtherFuelUse(kBtu),GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity(kgCO2e/ft2),DefaultData,ComplianceStatus,City Council Districts,SPD Beats,Zip Codes,Latitude,Longitude,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2015,NonResidential,Hotel,MAYFLOWER PARK HOTEL,659000030,7,DOWNTOWN,1927,1.0,12.0,88434,0,88434,Hotel,Hotel,88434.0,NaN,NaN,NaN,NaN,NaN,65.0,78.9,80.3,173.5,175.1,6981428.0,7097539.0,2023032.0,1080307.0,3686160.0,12724.0,1272388.0,0.0,249.43,2.64,No,Compliant,NaN,31.0,18081.0,47.61219025,-122.33799744,NaN,NaN
1,2,2015,NonResidential,Hotel,PARAMOUNT HOTEL,659000220,7,DOWNTOWN,1996,1.0,11.0,103566,15064,88502,"Hotel, Parking, Restaurant",Hotel,83880.0,Parking,15064.0,Restaurant,4622.0,NaN,51.0,94.4,99.0,191.3,195.2,8354235.0,8765788.0,0.0,1144563.0,3905411.0,44490.0,4448985.0,0.0,263.51,2.38,No,Compliant,NaN,31.0,18081.0,47.61310583,-122.33335756,NaN,NaN
2,3,2015,NonResidential,Hotel,WESTIN HOTEL,659000475,7,DOWNTOWN,1969,1.0,41.0,961990,0,961990,"Hotel, Parking, Swimming Pool",Hotel,757243.0,Parking,100000.0,Swimming Pool,0.0,NaN,18.0,96.6,99.7,242.7,246.5,73130656.0,75506272.0,19660404.0,14583930.0,49762435.0,37099.0,3709900.0,0.0,2061.48,1.92,Yes,Compliant,NaN,31.0,18081.0,47.61334897,-122.33769944,NaN,NaN
4,8,2015,NonResidential,Hotel,WARWICK SEATTLE HOTEL,659000970,7,DOWNTOWN,1980,1.0,18.0,119890,12460,107430,"Hotel, Parking, Swimming Pool",Hotel,123445.0,Parking,68009.0,Swimming Pool,0.0,NaN,67.0,120.1,122.1,228.8,227.1,14829099.0,15078243.0,0.0,1777841.0,6066245.0,87631.0,8763105.0,0.0,507.70,4.02,No,Compliant,NaN,31.0,19576.0,47.6137544,-122.3409238,NaN,NaN
5,9,2015,Nonresidential COS,Other,WEST PRECINCT (SEATTLE POLICE),660000560,7,DOWNTOWN,1999,1.0,2.0,97288,37198,60090,Police Station,Police Station,88830.0,NaN,NaN,NaN,NaN,NaN,NaN,135.7,146.9,313.5,321.6,12051984.0,13045258.0,0.0,2130921.0,7271004.0,47813.0,4781283.0,0.0,304.62,2.81,No,Compliant,NaN,7.0,19576.0,47.6164389,-122.33676431,NaN,NaN


### Exportation du dataframe

In [ ]:
#df_tot.to_csv('df_tot.csv', sep = '\t')